In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
!pip3 install thriftpy==0.3.9
!pip3 install thrift==0.9.3
!pip3 install thrift-sasl==0.2.1
!pip3 install sasl==0.2.1
!pip3 install six
!pip3 install bit_array
!pip3 install impyla
from impyla.dbapi import connect

Exception:
Traceback (most recent call last):
  File "/home/fdans/venv/lib/python3.5/site-packages/pip/_internal/basecommand.py", line 228, in main
    status = self.run(options, args)
  File "/home/fdans/venv/lib/python3.5/site-packages/pip/_internal/commands/install.py", line 291, in run
    resolver.resolve(requirement_set)
  File "/home/fdans/venv/lib/python3.5/site-packages/pip/_internal/resolve.py", line 103, in resolve
    self._resolve_one(requirement_set, req)
  File "/home/fdans/venv/lib/python3.5/site-packages/pip/_internal/resolve.py", line 262, in _resolve_one
    check_dist_requires_python(dist)
  File "/home/fdans/venv/lib/python3.5/site-packages/pip/_internal/utils/packaging.py", line 46, in check_dist_requires_python
    feed_parser.feed(metadata)
  File "/usr/lib/python3.5/email/feedparser.py", line 178, in feed
    self._input.push(data)
  File "/usr/lib/python3.5/email/feedparser.py", line 104, in push
    self._partial.write(data)
TypeError: string argument expecte

ImportError: No module named 'impyla'

In [ ]:
hive_conn = connect(host='analytics1003.eqiad.wmnet', port=10000, auth_mechanism='PLAIN')

In [ ]:
hive_cursor = hive_conn.cursor()

In [ ]:
hive_cursor.execute('SELECT user_agent, count(*) as pageviews FROM wmf.webrequest WHERE year=2018 and month=4 and day=15 and hour=12 and is_pageview=true group by user_agent')

In [ ]:
all_the_rows = hive_cursor.fetchall()
print(all_the_rows[0])

In [ ]:
print("Total requests (number coming from manual hive query): 384,683,254")

In [ ]:
total_pageviews = sum(row[1] for row in all_the_rows)
print("Total pageviews: " + str(total_pageviews))

In [ ]:
!pip install ua-parser==0.7.3
print(user_agent_parser.Parse("%E6%AC%A7%E8%B7%AF%E8%AF%8D%E5%85%B8/8.7.0.4 CFNetwork/894 Darwin/17.4.0"))

In [ ]:
# OLD UAPARSER
!pip install ua-parser==0.7.3
from ua_parser import user_agent_parser
old_values = []
for row in all_the_rows:
    old_values.append((user_agent_parser.Parse(row[0]), row[1]))

In [ ]:
!pip install ua-parser==0.8.0
from ua_parser import user_agent_parser
print(user_agent_parser.Parse("Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0)"))

In [ ]:
# NEW UAPARSER
!pip install ua-parser==0.8.0
from ua_parser import user_agent_parser
new_values = []
for row in all_the_rows:
    new_values.append((user_agent_parser.Parse(row[0]), row[1]))

In [ ]:
differing_pageview_useragents = []
differing_pageviews = 0
for i, row_old in enumerate(old_values):
    row_new = new_values[i]
    if (row_old != row_new):
        differing_pageviews += row_old[1]
        differing_pageview_useragents.append((row_old[0], row_new[0], row_old[1]))
print("User Agent strings that are parsed differently: " + str(len(differing_pageview_useragents)))
print("Pageviews whose UA is parsed differently: " + str(differing_pageviews))
print("Percentage: " + str(differing_pageviews / total_pageviews *100))

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
print("This is the OS family distribution of the differing pageviews, according to the old UA Parser:")
pp.pprint(getDistributionFor('os', 'family'))

In [ ]:
print("This is the OS family distribution of the differing pageviews, according to the NEW UA Parser:")
pp.pprint(getNewDistributionFor('os', 'family'))

In [ ]:
print("This is the OS family distribution of the differing pageviews, according to the old UA Parser:")
pp.pprint(getDistributionForExcludingWindows('device', 'brand'))

In [ ]:
print(pp.pprint(getMovementFromIE7()))

In [ ]:
pp.pprint(getIE7MovedUAStrings())

In [ ]:
def getDistributionFor(uafield, uasubfield):
    distribution = {}
    for i,row_old in enumerate(old_values):
        views = row_old[1]
        row_new = new_values[i]
        k = str(row_old[0][uafield][uasubfield])
        if (row_old != row_new):
            if k not in distribution:
                distribution[k] = views
            else:
                distribution[k] += views
    return distribution

In [ ]:
def getNewDistributionFor(uafield, uasubfield):
    distribution = {}
    for i,row_old in enumerate(old_values):
        views = row_old[1]
        row_new = new_values[i]
        k = str(row_new[0][uafield][uasubfield])
        if (row_old != row_new):
            if k not in distribution:
                distribution[k] = views
            else:
                distribution[k] += views
    return distribution

In [ ]:
def getDistributionForExcludingWindows(uafield, uasubfield):
    distribution = {}
    for i,row_old in enumerate(old_values):
        views = row_old[1]
        row_new = new_values[i]
        k = str(row_old[0][uafield][uasubfield])
        if (row_old != row_new and row_new[0]['os']['family'] != 'Windows'):
            if k not in distribution:
                distribution[k] = views
            else:
                distribution[k] += views
    return distribution

In [ ]:
def getMovementFromIE7():
    distribution = {}
    for i,row_old in enumerate(old_values):
        views = row_old[1]
        row_new = new_values[i]
        k = str(row_old[0]['user_agent']['family']) + " " + str(row_old[0]['user_agent']['major']) + "=>" + str(row_new[0]['user_agent']['family']) + " " + str(row_new[0]['user_agent']['major'])
        if (row_old != row_new and row_new[0]['user_agent']['family'] == 'IE'):
            if k not in distribution:
                distribution[k] = views
            else:
                distribution[k] += views
    return distribution

In [ ]:
def getIE7MovedUAStrings():
    distribution = {}
    for i,row_old in enumerate(old_values):
        views = row_old[1]
        row_new = new_values[i]
        k = str(row_old[0]['string'])
        if (row_old != row_new and row_old[0]['user_agent']['family'] == 'IE' and row_old[0]['user_agent']['major'] == '7' and row_new[0]['user_agent']['major'] == '11'):
            if k not in distribution:
                distribution[k] = views
            else:
                distribution[k] += views
    return distribution

In [ ]:
import re
def is_bot(device_family, user_agent):
    bot_ua_pattern = re.compile('(.*(bot|spider|WordPress|AppEngine|AppleDictionar'
                            'yService|Python-urllib|python-requests|Google-HTT'
                            'P-Java-Client|[Ff]acebook|[Yy]ahoo|RockPeaks|http'
                            ').*|(goo wikipedia|MediaWikiCrawler-Google|wikiwi'
                            'x-bot|Java|curl|PHP|Faraday|HTTPC|Ruby|\.NET|Pyth'
                            'on|Apache|Scrapy|PycURL|libwww|Zend|wget|nodemw|W'
                            'inHttpRaw|Twisted|com\.eusoft|Lagotto|Peggo|Recuw'
                            'eb|check_http|Magnus|MLD|Jakarta|find-link|J\. Ri'
                            'ver|projectplan9|ADmantX|httpunit|LWP|iNaturalist'
                            '|WikiDemo|FSResearchIt|livedoor|Microsoft Monitor'
                            'ing|MediaWiki|User:|User_talk:|github|tools.wmfla'
                            'bs.org|Blackboard Safeassign|Damn Small XSS|\S+@'
                            '\S+\.[a-zA-Z]{2,3}).*)$')
    """
    Tests the raw user agent string against a bot regular expression
    if uaparser isn't already marking it as a spider
    """
    if device_family == 'Spider':
        return True
    elif device_family == 'Other':
        ua_string = user_agent.strip('"')
        return bool(bot_ua_pattern.match(ua_string))
    return False

In [ ]:
!pip install ua-parser==0.7.3
for row in all_the_rows:
    print(row)
    break